In [1]:
!pip install pdfminer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 33.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 72.7 MB/s eta 0:00:00
  Created wheel for pdfminer: filename=pdfminer-20191125-py3-none-any.whl size=6140095 sha256=2744f696df771cc81b269c461bfeb6505fdca9712d49aaa3365e82e6256d1340
  Stored in directory: /root/.cache/pip/wheels/4e/c1/68/f7bd0a8f514661f76b5cbe3b5f76e0033d79f1296012cbbf72
Successfully built pdfminer


In [2]:
pip install -U sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=2b3de8f837ea6dba6eb396545b771438b5260f5e8d70bd0173612dffb0889ba5
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [8]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
import re, string
from string import punctuation
import logging

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

lemmatizer = WordNetLemmatizer()

# Configure the logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Create the logger
logger = logging.getLogger(__name__)

# Set the log level to INFO
logger.setLevel(logging.INFO)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [9]:
def extract_text_from_pdf(file_path):
    resource_manager = PDFResourceManager()
    return_str = StringIO()
    layout_params = LAParams()
    device = TextConverter(resource_manager, return_str, laparams=layout_params)
    interpreter = PDFPageInterpreter(resource_manager, device)

    fp = open(file_path, 'rb')

    password = ""
    max_pages = 0
    caching = True
    page_nos = set()
    for page in PDFPage.get_pages(fp, page_nos, maxpages=max_pages, password=password, caching=caching,
                                    check_extractable=True):
        interpreter.process_page(page)

    text = return_str.getvalue().lower()
    return text


def remove_punctuation(text):
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    return cleaned_text


def remove_specific_words(text):
    words_to_remove = ['gender', 'skills', 'tools', 'date of birth', 'dob', 'languages', 'known',
                       'email', 'mobile', 'phone', 'no', 'linkedin', 'github', 'profile', 'male', 'female',
                       'english', 'tamil', 'hindi', 'address', 'objective', 'intern', 'full', 'time', 'academic',
                       'qualification', 'skill set', 'skill', 'languages', 'platform', 'technology', 'technologies',
                       'areas of interest', 'record', '\n', '|', 'link', '\t',
                       ]
    pattern = r'\b(?:{})\b'.format('|'.join(words_to_remove))
    cleaned_text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    return cleaned_text


def remove_stop_words(text):
    stop_words = set(stopwords.words("english"))
    punctuation = list(string.punctuation)
    stop_words.update(punctuation)

    text = " ".join(x for x in text.split() if x not in stop_words)
    return text


def remove_personal_info(text):
    # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)
    # Remove phone numbers
    text = re.sub(r'\d{3}[-\.\s]?\d{3}[-\.\s]?\d{4}', '', text)
    # Remove addresses
    text = re.sub(r'\d+\s+[\w\s]+,\s+[\w\s]+,\s+[\w\s]+', '', text)
    return text


def remove_numbers(text):
    text = re.sub(r'\d','',text)
    return text


def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])


def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


def encoder(text, window_size=384):
    embeddings = list()
    counter = 0

    while len(text) / window_size > counter:
        encoded_input = tokenizer(text[window_size * counter : window_size * (counter+1)], padding=True, truncation=True, return_tensors='pt')

        with torch.no_grad():
            model_output = model(**encoded_input)

        sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

        counter += 1
        embeddings.append(sentence_embeddings)

    return embeddings


def preprocess_encode_text(text):
    text = remove_personal_info(text)
    logger.info(f"Length of Text after removing personal data is {len(text)}")
    text = remove_specific_words(text)
    logger.info(f"Length of Text after removing technical words from data is {len(text)}")
    text = remove_stop_words(text)
    logger.info(f"Length of Text after removing stop words from data is {len(text)}")
    text = remove_punctuation(text)
    logger.info(f"Length of Text after removing punctuation from data is {len(text)}")
    text = remove_numbers(text)
    logger.info(f"Length of Text after removing numbers from data is {len(text)}")
    text = lemmatize_text(text)
    logger.info(f"Length of Text after Lemmatization from data is {len(text)}")

    logger.info(f"Text after Lemmatization is: \n{text}")

    return encoder(text)


In [10]:
# extract_text_from_pdf("/content/Sriram S's Resume-hackerresume.pdf")
text = extract_text_from_pdf("/content/Sriram S's Resume-hackerresume.pdf")
len(text)

2134

In [11]:
embeddings = preprocess_encode_text(text)

INFO:__main__:Length of Text after removing personal data is 2093
INFO:__main__:Length of Text after removing technical words from data is 2003
INFO:__main__:Length of Text after removing stop words from data is 1712
INFO:__main__:Length of Text after removing punctuation from data is 1647
INFO:__main__:Length of Text after removing numbers from data is 1624
INFO:__main__:Length of Text after Lemmatization from data is 1585
INFO:__main__:Text after Lemmatization is: 
viswanathan layout lakshmipuram coimbatore leetcode sriram education psg college msc data sciencecgpa coimbatoreseptember present programming librariesframeworks platform database python sql r c scikitlearn seaborn plotly flask mediapipewindows linux tableaumysql mongodb elastic search project opensource parliamentconnect semanticsearchengine python flask elastic search sentence transformerssbert used encode parliament question answer bulkinserted elasticsearch e cient retrieval question answer smart indiahackathon yoga la

In [12]:
len(embeddings)

5

In [13]:
embeddings

[tensor([[ 3.9823e-02,  2.5041e-02, -3.4360e-02,  9.2012e-05, -3.4988e-03,
          -1.6369e-02, -1.3146e-02,  9.0131e-03, -2.4278e-02, -9.8238e-03,
           5.6035e-02,  2.9018e-02,  8.6572e-03,  1.1661e-01,  3.2006e-02,
          -9.6222e-02,  1.2090e-02,  1.5003e-02, -4.4431e-02,  1.6519e-02,
          -4.5617e-02,  2.9141e-02, -1.4877e-02,  1.9722e-02,  3.7375e-02,
           2.2797e-02, -2.0012e-02, -7.2761e-03,  1.1910e-02, -7.0501e-02,
           6.0656e-03, -1.2878e-03, -1.5662e-02,  2.8770e-02,  2.4315e-06,
          -2.3223e-02, -1.9962e-02, -1.5690e-02,  1.9507e-04,  2.6582e-02,
           4.7962e-02,  4.0733e-03,  2.3822e-02,  8.9517e-03, -2.2252e-02,
           2.2179e-03,  4.1268e-02, -6.6880e-02,  3.4698e-02,  5.5729e-02,
          -9.7960e-03, -1.3521e-02, -5.3494e-02, -2.6185e-04,  1.2004e-02,
           2.6644e-03, -2.1828e-02, -1.6683e-02,  5.8161e-02,  1.2802e-02,
          -1.8752e-02,  7.0525e-03, -2.6729e-02, -4.5665e-03,  7.2922e-02,
           4.5381e-02, -3

In [14]:
job_description = '''
We are looking for highly motivated and uniquely talented Python Developers to join our Data Team. Our team includes a diverse range of technical backgrounds and areas of subject expertise and is responsible for the aggregation, transformation, analysis, and visualization of large amounts of data from the web, to help our customers make smarter data-driven decisions.


Are you someone with the creativity, technical skills, attention to detail, and enthusiasm to join our team and help leverage data to enable enterprises to grow their business.


Go ahead and apply if you are willing to work from Kochi office or looking to relocate to Kochi, Kerala.


Skills & Experience

Proficiency in Python.
Experience in using version control (preferably Git) in projects.
Basic understanding of web technologies (HTML, JavaScript etc.)
Good knowledge of databases
Familiarity with data processing tools (pandas, regex) will be an added advantage
Ability to work independently, prioritize tasks, and take initiative.
Ability to document requirements and specifications.
Excellent time-management, multi-tasking, communication, and interpersonal skills.
'''

In [15]:
embeddings_job = preprocess_encode_text(job_description)

INFO:__main__:Length of Text after removing personal data is 1149
INFO:__main__:Length of Text after removing technical words from data is 1108
INFO:__main__:Length of Text after removing stop words from data is 894
INFO:__main__:Length of Text after removing punctuation from data is 862
INFO:__main__:Length of Text after removing numbers from data is 862
INFO:__main__:Length of Text after Lemmatization from data is 851
INFO:__main__:Text after Lemmatization is: 
We looking highly motivated uniquely talented Python Developers join Data Team Our team includes diverse range technical background area subject expertise responsible aggregation transformation analysis visualization large amount data web help customer make smarter datadriven decision Are someone creativity technical attention detail enthusiasm join team help leverage data enable enterprise grow business Go ahead apply willing work Kochi office looking relocate Kochi Kerala Experience Proficiency Python Experience using versio

In [16]:
len(embeddings_job)

3

In [17]:
embeddings_job

[tensor([[ 3.9448e-02,  7.8907e-02, -1.9311e-02, -3.1318e-02,  4.7441e-02,
           2.0711e-02, -3.2682e-02, -4.7362e-02, -4.7775e-02, -2.9680e-02,
           4.3068e-02,  2.4568e-02, -1.3606e-02,  1.0174e-01, -5.5824e-04,
          -5.9628e-02, -1.7674e-02, -1.0509e-02, -5.3789e-04,  3.4643e-02,
          -2.6410e-02,  4.0998e-02, -2.4453e-02, -1.5994e-03, -3.4388e-04,
           3.3048e-02,  8.7689e-03,  2.0461e-02, -1.0711e-02, -1.3623e-02,
           4.9318e-02, -1.2008e-02, -1.3415e-02,  2.3014e-02,  1.9081e-06,
          -2.1754e-02, -2.6293e-02,  6.0036e-03, -2.2403e-02, -4.4352e-02,
           2.6215e-02, -2.6077e-02,  8.9965e-03,  2.8558e-02, -7.1204e-02,
          -3.3451e-03,  3.6709e-02, -7.2908e-02,  4.9883e-02,  4.8268e-02,
           7.2246e-03, -8.4248e-03,  2.6258e-03,  1.1633e-02, -4.6646e-04,
           9.8583e-03, -2.4483e-02, -1.4676e-02,  7.7081e-02, -3.5165e-02,
           6.8047e-02, -2.2161e-02, -2.3761e-02, -1.4744e-02,  5.0803e-02,
           6.1533e-02, -3

In [18]:
def preprocess_encode_text(text):
    text = remove_personal_info(text)
    logger.info(f"Length of Text after removing personal data is {len(text)}")
    text = remove_specific_words(text)
    logger.info(f"Length of Text after removing technical words from data is {len(text)}")
    text = remove_stop_words(text)
    logger.info(f"Length of Text after removing stop words from data is {len(text)}")
    text = remove_punctuation(text)
    logger.info(f"Length of Text after removing punctuation from data is {len(text)}")
    text = remove_numbers(text)
    logger.info(f"Length of Text after removing numbers from data is {len(text)}")
    text = lemmatize_text(text)
    logger.info(f"Length of Text after Lemmatization from data is {len(text)}")

    logger.info(f"Text after Lemmatization is: \n{text}")

    return encoder(text)


def encoder(text, window_size=384, stride=128):
    embeddings = []
    counter = 0

    while len(text) > window_size * counter:
        window_text = text[window_size * counter : window_size * (counter + 1)]
        window_embeddings = encode_window(window_text)
        embeddings.append(window_embeddings)
        counter += stride

    return embeddings

def encode_window(window_text):
    encoded_input = tokenizer(window_text, padding=True, truncation=True, return_tensors='pt')

    with torch.no_grad():
        model_output = model(**encoded_input)

    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    return sentence_embeddings


In [19]:
preprocessed_encoded_text = preprocess_encode_text(job_description)


INFO:__main__:Length of Text after removing personal data is 1149
INFO:__main__:Length of Text after removing technical words from data is 1108
INFO:__main__:Length of Text after removing stop words from data is 894
INFO:__main__:Length of Text after removing punctuation from data is 862
INFO:__main__:Length of Text after removing numbers from data is 862
INFO:__main__:Length of Text after Lemmatization from data is 851
INFO:__main__:Text after Lemmatization is: 
We looking highly motivated uniquely talented Python Developers join Data Team Our team includes diverse range technical background area subject expertise responsible aggregation transformation analysis visualization large amount data web help customer make smarter datadriven decision Are someone creativity technical attention detail enthusiasm join team help leverage data enable enterprise grow business Go ahead apply willing work Kochi office looking relocate Kochi Kerala Experience Proficiency Python Experience using versio

In [20]:
preprocessed_encoded_text

[tensor([[ 3.9448e-02,  7.8907e-02, -1.9311e-02, -3.1318e-02,  4.7441e-02,
           2.0711e-02, -3.2682e-02, -4.7362e-02, -4.7775e-02, -2.9680e-02,
           4.3068e-02,  2.4568e-02, -1.3606e-02,  1.0174e-01, -5.5824e-04,
          -5.9628e-02, -1.7674e-02, -1.0509e-02, -5.3789e-04,  3.4643e-02,
          -2.6410e-02,  4.0998e-02, -2.4453e-02, -1.5994e-03, -3.4388e-04,
           3.3048e-02,  8.7689e-03,  2.0461e-02, -1.0711e-02, -1.3623e-02,
           4.9318e-02, -1.2008e-02, -1.3415e-02,  2.3014e-02,  1.9081e-06,
          -2.1754e-02, -2.6293e-02,  6.0036e-03, -2.2403e-02, -4.4352e-02,
           2.6215e-02, -2.6077e-02,  8.9965e-03,  2.8558e-02, -7.1204e-02,
          -3.3451e-03,  3.6709e-02, -7.2908e-02,  4.9883e-02,  4.8268e-02,
           7.2246e-03, -8.4248e-03,  2.6258e-03,  1.1633e-02, -4.6646e-04,
           9.8583e-03, -2.4483e-02, -1.4676e-02,  7.7081e-02, -3.5165e-02,
           6.8047e-02, -2.2161e-02, -2.3761e-02, -1.4744e-02,  5.0803e-02,
           6.1533e-02, -3

In [21]:
len(preprocessed_encoded_text)

1

In [22]:
len(preprocessed_encoded_text[0][0])

768

In [29]:
# extract_text_from_pdf("/content/Sriram S's Resume-hackerresume.pdf")
text = extract_text_from_pdf("/content/Sriram S's Resume-hackerresume.pdf")
len(text)

embeddings = preprocess_encode_text(text)

INFO:__main__:Length of Text after removing personal data is 2093
INFO:__main__:Length of Text after removing technical words from data is 2003
INFO:__main__:Length of Text after removing stop words from data is 1712
INFO:__main__:Length of Text after removing punctuation from data is 1647
INFO:__main__:Length of Text after removing numbers from data is 1624
INFO:__main__:Length of Text after Lemmatization from data is 1585
INFO:__main__:Text after Lemmatization is: 
viswanathan layout lakshmipuram coimbatore leetcode sriram education psg college msc data sciencecgpa coimbatoreseptember present programming librariesframeworks platform database python sql r c scikitlearn seaborn plotly flask mediapipewindows linux tableaumysql mongodb elastic search project opensource parliamentconnect semanticsearchengine python flask elastic search sentence transformerssbert used encode parliament question answer bulkinserted elasticsearch e cient retrieval question answer smart indiahackathon yoga la

In [30]:
len(embeddings[0][0])

768

In [31]:
embeddings

[tensor([[ 3.9823e-02,  2.5041e-02, -3.4360e-02,  9.2012e-05, -3.4988e-03,
          -1.6369e-02, -1.3146e-02,  9.0131e-03, -2.4278e-02, -9.8238e-03,
           5.6035e-02,  2.9018e-02,  8.6572e-03,  1.1661e-01,  3.2006e-02,
          -9.6222e-02,  1.2090e-02,  1.5003e-02, -4.4431e-02,  1.6519e-02,
          -4.5617e-02,  2.9141e-02, -1.4877e-02,  1.9722e-02,  3.7375e-02,
           2.2797e-02, -2.0012e-02, -7.2761e-03,  1.1910e-02, -7.0501e-02,
           6.0656e-03, -1.2878e-03, -1.5662e-02,  2.8770e-02,  2.4315e-06,
          -2.3223e-02, -1.9962e-02, -1.5690e-02,  1.9507e-04,  2.6582e-02,
           4.7962e-02,  4.0733e-03,  2.3822e-02,  8.9517e-03, -2.2252e-02,
           2.2179e-03,  4.1268e-02, -6.6880e-02,  3.4698e-02,  5.5729e-02,
          -9.7960e-03, -1.3521e-02, -5.3494e-02, -2.6185e-04,  1.2004e-02,
           2.6644e-03, -2.1828e-02, -1.6683e-02,  5.8161e-02,  1.2802e-02,
          -1.8752e-02,  7.0525e-03, -2.6729e-02, -4.5665e-03,  7.2922e-02,
           4.5381e-02, -3

In [32]:
import torch.nn.functional as F
import torch

def cosine_similarity(encoded_text1, encoded_text2):
    # Convert lists to tensors
    encoded_text1 = torch.cat(encoded_text1, dim=0)
    encoded_text2 = torch.cat(encoded_text2, dim=0)

    # Calculate the cosine similarity
    similarity = F.cosine_similarity(encoded_text1, encoded_text2, dim=1)

    return similarity



In [33]:
score_sriram = cosine_similarity(embeddings,preprocessed_encoded_text)
score_sriram

tensor([0.4083])

In [34]:
text = extract_text_from_pdf("/content/20pd17.pdf")

embeddings_shimer = preprocess_encode_text(text)

INFO:__main__:Length of Text after removing personal data is 3228
INFO:__main__:Length of Text after removing technical words from data is 3011
INFO:__main__:Length of Text after removing stop words from data is 2486
INFO:__main__:Length of Text after removing punctuation from data is 2389
INFO:__main__:Length of Text after removing numbers from data is 2294
INFO:__main__:Length of Text after Lemmatization from data is 2223
INFO:__main__:Text after Lemmatization is: 
muhammed shimer spd th july comshimer compounder street pollachi nadu obtain position period six month may november currently pursuing rd year year integrated msc data science thedepartment applied mathematics computational science psg college object oriented programming supervised unsupervised learning msc data science psg college coimbatore xii higher secondary state board pkd matric hr sec school pollachi x sslc state board pkd matric hr sec school pollachi non project database management system cgpa analysebi web appli

In [35]:
score_shimer = cosine_similarity(embeddings_shimer,preprocessed_encoded_text)
score_shimer

tensor([0.4141])

In [ ]:
# We can see that shimer's resume is more apt to the job description provided. So the company can shortlist shimer for further rounds.